In [2]:
import h5py
import numpy as np
from tqdm import tqdm
import copy
from torch.utils.data import Dataset

c:\Users\pc\anaconda3\envs\Othello_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Method to load the game log from an HDF5 file
def load_game_log(file_path):
    # file_path: path to the HDF5 file containing the game log
    h5f = h5py.File(file_path, 'r')  # Open the HDF5 file in read mode
    game_name = file_path.split('/')[-1].replace(".h5", "")  # Extract the game name from the file path
    game_log = np.array(h5f[game_name][:])  # Read the game log data as a NumPy array
    h5f.close()  # Close the HDF5 file
    return game_log  # Return the loaded game log

In [4]:
game_log  = load_game_log('./dataset/16740.h5')  

In [5]:
print(len(game_log))
print(len(game_log[0]))
print(len(game_log[1]))
print(game_log.shape)
print(game_log[0,0])      # plateau initial
print(game_log[1,0]) 

2
60
60
(2, 60, 8, 8)
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  1 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


In [ ]:
print(game_log.shape)
print(game_log[0,0])      
print(game_log[1,0]) 
print(game_log[0,1])      # premier coup
print(game_log[1,1])

(2, 60, 8, 8)
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  1 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  1  1  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


In [11]:
print(game_log[0,1])      # premier coup

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  1  1  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


In [17]:
def print_board(board):
    """
    board : array 8x8 avec valeurs -1, 0, +1
    Affiche le plateau avec des symboles jolis.
    """
    for i in range(8):
        row = ""
        for j in range(8):
            v = board[i, j]
            if v == 1:
                row += " W"   # White
            elif v == -1:
                row += " B"  # Black
            else:
                row += " ."
        print(row)
    print()  # ligne vide à la fin


In [18]:
def inspect_step(game_log, t):
    """
    game_log : array (2, 60, 8, 8)
    t : index du coup (0 à 58 si tu veux voir t et t+1)
    """
    boards = game_log[0]  # (60, 8, 8)
    moves  = game_log[1]  # (60, 8, 8)

    print(f"=== Étape t = {t} ===")

    print("Plateau AVANT le coup (game_log[0, t]) :")
    print_board(boards[t])

    print("Coup joué (game_log[1, t]) : (un seul '1') :")
    print(moves[t])
    print()

    if t + 1 < boards.shape[0]:
        print("Plateau APRÈS le coup (game_log[0, t+1]) :")
        print_board(boards[t+1])
    else:
        print("t+1 dépasse la fin de la partie (t = 59 est le dernier index).")


In [20]:
game_log = load_game_log("dataset/16740.h5")

inspect_step(game_log, 0)   # regarde le tout premier coup
inspect_step(game_log, 1)   # regarde le deuxième coup
inspect_step(game_log, 10)  # plus loin dans la partie


=== Étape t = 0 ===
Plateau AVANT le coup (game_log[0, t]) :
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . B W . . .
 . . . W B . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .

Coup joué (game_log[1, t]) : (un seul '1') :
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

Plateau APRÈS le coup (game_log[0, t+1]) :
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . B W . . .
 . . . W W . . .
 . . . . W . . .
 . . . . . . . .
 . . . . . . . .

=== Étape t = 1 ===
Plateau AVANT le coup (game_log[0, t]) :
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . B W . . .
 . . . W W . . .
 . . . . W . . .
 . . . . . . . .
 . . . . . . . .

Coup joué (game_log[1, t]) : (un seul '1') :
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

Plateau APRÈS le coup (gam

In [26]:
filelist = "..\\deeplearning2playothello\\train.txt"
with open(filelist) as f:
    list_files = [line.rstrip() for line in f]
print(list_files[:5])  # affiche les 5 premiers fichiers du dataset

['1000029.h5', '100009.h5', '1000217.h5', '1000240.h5', '1000970.h5']


In [27]:
print(len(list_files))  # affiche le nombre total de fichiers dans le dataset

6001
